# Technical Analysis

In [155]:
import polars as pl
import pyarrow.parquet as pq
import polars.selectors as cs
import numpy as np
import decimal 

lf_prices = pl.scan_parquet("data/prices_pool.parquet")
lf_liquidity = pl.scan_parquet("data/liquidity_pool.parquet")

In [175]:
# create open, high, low, close and volume measures as the basis for technical analysis 

one_minute = "60i"
one_hour = "360i"
one_day = "8640i"
intervals = one_hour

usdc_address = "a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"

ohlc = lf_prices.filter(
    pl.any_horizontal(
        pl.col("token0_address") == usdc_address, 
        pl.col("token1_address") == usdc_address,
    )
).sort(
    ["block_number", "transaction_index", "log_index"]
).with_columns([
    pl.col("block_number").cast(pl.Int64),
    pl.struct(["amount0", "token0_decimals"]).map_batches(
        lambda a: int.from_bytes(a.struct.field("amount0")[0], "little") / (10 ** a.struct.field("token0_decimals")[0])
    ).alias("amount0"),
    pl.struct(["amount1", "token1_decimals"]).map_batches(
        lambda a: int.from_bytes(a.struct.field("amount1")[0], "little") / (10 ** a.struct.field("token1_decimals")[0])
    ).alias("amount1"),
]).set_sorted("block_number").group_by_dynamic(
    index_column="block_number",
    every=intervals,
    by="pool_address",
).agg([
    pl.col("price").first().alias("OPEN"),
    pl.col("price").max().alias("HIGH"),
    pl.col("price").min().alias("LOW"),
    pl.col("price").last().alias("CLOSE"),
    pl.col("timestamp").min().alias("start_time"),
    pl.col("timestamp").max().alias("end_time"),
    pl.col("amount0").sum().alias("volume0"),
    pl.col("amount1").sum().alias("volume1"),
    pl.col("token0_address").first(),
    pl.col("token0_name").first(),
    pl.col("token0_symbol").first(),
    pl.col("token1_address").first(),
    pl.col("token1_name").first(),
    pl.col("token1_symbol").first(),
    pl.col("tick_spacing").first(),
    pl.col("fee").first(),
]).collect()

ohlc

arrow_table = ohlc.to_arrow()
pq.write_table(arrow_table, "data/ohlc.parquet", compression="zstd")
del arrow_table
del ohlc

lf_ohlc = pl.scan_parquet("data/ohlc.parquet")

In [187]:
lf_ohlc.select([
    pl.col("token0_symbol"),
    pl.col("token0_address"),
    pl.col("token1_symbol"),
    pl.col("pool_address"),
    pl.col("OPEN"),
    pl.col("HIGH"),
    pl.col("LOW"),
    pl.col("CLOSE"),
    pl.col("volume0"),
    pl.col("volume1"),
]).collect().glimpse()


SyntaxError: invalid syntax (1018223623.py, line 5)